In [1]:
import mnist
import torch
import torch.nn as nn
import torch.tensor as tensor
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
x_train, t_train, x_test, t_test = mnist.load()
xtrain=tensor(x_train/255,dtype=torch.float)
ttrain=tensor(t_train,dtype=torch.int64)
xtest=tensor(x_test/255,dtype=torch.float)
ttest=tensor(t_test,dtype=torch.int64)

plt.imshow(xtrain[1].view(28,28))
xtrain=xtrain.view(-1,1,28,28)
xtest=xtest.view(-1,1,28,28)
print(xtrain.shape,xtest.shape)

torch.Size([60000, 1, 28, 28]) torch.Size([10000, 1, 28, 28])


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(256, 120)#(28-4)/2=12,(12-4)/2=4(size=4*4), there are 16 filters
        self.fc2 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,256)
        x = torch.relu(self.fc1(x))
        #x=F.softmax(self.fc2(x),dim=1)
        x=self.fc2(x)
        return x
net = Net()

In [3]:
criterion = torch.nn.CrossEntropyLoss() # mean square error loss
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
Loss=0
from time import time
stime=time()
tsize=xtrain.shape[0]
bsize=30 #******************* 300 or 2000 result bad
for epoch in range(1):
    for i in range(int(tsize/bsize)):
            x=xtrain[bsize*i:bsize*(i+1)]
            t=ttrain[bsize*i:bsize*(i+1)]
            
            optimizer.zero_grad()
            y = net(x)
            loss = criterion(y, t)
            #print('epoch: ', epoch, 'loss:', loss.item())
            loss.backward()
            optimizer.step()
print(loss.item())
print(time()-stime)


0.10044614970684052
8.59053659439087


In [4]:


ypred=net(xtest)

print(ypred.max(1)[1])
print(ttest)

tensor([7, 2, 1,  ..., 4, 5, 6])
tensor([7, 2, 1,  ..., 4, 5, 6])


In [5]:
cmp=ttest.eq(ypred.max(1)[1])
true=len([x for x in cmp if x==1])
false=len([x for x in cmp if x==0])
total=true+false
print(true/total)

0.9295
